# PowerModels.jl: Load PSS/E RAW, Run Power Flow, and Compute PTDF

1. Load PSS/E RAW
2. Run AC LF
3. Run DC LF
4. Compute PTDF


In [15]:
using Pkg
Pkg.activate(".")
test_system_dir = "./Test System/" # The system path goes here
raw_path = test_system_dir * "SmallSystem_case.raw"

  Activating new project at `~/github/choose-solver`


"./Test System/SmallSystem_case.raw"

## 1. Load PSS/E RAW

In [16]:
using PowerModels, Ipopt
data = PowerModels.parse_file(raw_path; import_all=true)

[info | PowerModels]: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## 2. Run AC Power Flow

In [17]:
@time begin
    # Method 1: AC power flow (Ipopt recommended)
    ac_pf = PowerModels.solve_ac_pf(data, Ipopt.Optimizer)
    # ac_pf
end

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:    98537
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   282390

Total number of variables............................:    14708
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    14661
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.10e+01 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.868606
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.0
  "solution"           => Dict{String, Any}("baseMVA"=>100.0, "gen"=>Dict{Strin…
  "objective_lb"       => -Inf

In [18]:
@time begin
    # Method 2: AC power flow (Ipopt recommended)
    ac_pf = PowerModels.compute_ac_pf(data)
    # ac_pf
end

  0.156001 seconds (868.30 k allocations: 154.985 MiB)


Dict{String, Any} with 5 entries:
  "optimizer"          => "NLsolve"
  "termination_status" => true
  "objective"          => 0.0
  "solution"           => Dict{String, Any}("gen"=>Dict{String, Any}("153"=>Dic…
  "solve_time"         => 0.103268

## 3. Run DC Power Flow

In [19]:
@time begin
    # DC power flow (linear approximation)
    dc_pf = PowerModels.solve_dc_pf(data, Ipopt.Optimizer)
    dc_pf
end

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:    25283
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     7354
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     7354
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.10e+01 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.0126069
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.0
  "solution"           => Dict{String, Any}("baseMVA"=>100.0, "gen"=>Dict{Strin…
  "objective_lb"       => -Inf

## 4. Compute a PTDF matrix (DC approximation)
The result has one row per branch and one column per bus.

In [20]:
@time begin
    basic = PowerModels.make_basic_network(data)
    PTDF = PowerModels.calc_basic_ptdf_matrix(basic)   # n_branch × n_bus (sparse)
    size(PTDF)
end

[info | PowerModels]: updated generator 1 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 519 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 599 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 491 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 228 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 332 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 190 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 227 cost function with order 2 to a function of order 3: [0.0, 100.0, 0.0]
[info | PowerModels]: updated generator 297 cost function with order 2 to a functi

(9140, 6717)

## 5. Run Contingency Sensitivity Computation

Compute PTDF matrices for base case + 500 contingencies.
This matches the configuration in the PowSyBl notebook (500 contingencies, 1000 monitored branches).

In [21]:
# Get network data
data = PowerModels.parse_file(raw_path; import_all=true)
basic = PowerModels.make_basic_network(data)

# Configuration matching PowSyBl notebook
num_contingencies = 500
num_monitored = 1000

# Get sorted branch IDs for deterministic selection
all_branch_ids = sort(collect(keys(basic["branch"])))
contingency_branches = all_branch_ids[1:min(num_contingencies, length(all_branch_ids))]
monitored_branches = all_branch_ids[1:min(num_monitored, length(all_branch_ids))]

println("Configuration:")
println("  Contingencies: $(length(contingency_branches))")
println("  Monitored branches: $(length(monitored_branches))")

[info | PowerModels]: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in GENERATOR are missing: O2, F2, O3, F3, O4, F4, WMOD, WPF
[warn | PowerModels]: The following fields in

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [22]:
@time begin
    # Compute PTDF for base case
    base_ptdf = PowerModels.calc_basic_ptdf_matrix(basic)
    
    # Compute PTDF for each contingency (using only 500 contingencies to match PowSyBl)
    successful = 0
    for (i, br_id) in enumerate(contingency_branches)
        try
            # Create contingency by disabling the branch
            basic_cont = deepcopy(basic)
            basic_cont["branch"][br_id]["br_status"] = 0  # outaged
            
            # Recompute PTDF for this contingency
            ptdf_cont = PowerModels.calc_basic_ptdf_matrix(basic_cont)
            successful += 1
        catch e
            # Some contingencies may cause infeasibility - this is expected
        end
    end
    println("Successfully computed PTDF for $successful/$(length(contingency_branches)) contingencies")
end

[error | PowerModels]: Failed factorization in calc_susceptance_matrix_inv
Successfully computed PTDF for 499/500 contingencies
283.321727 seconds (415.89 M allocations: 1.083 TiB, 10.49% gc time, 0.00% compilation time)
